## Octoparse API 串接示範
使用 `KGBuilder` 從 Octoparse 取得爬蟲任務的爬取結果

1. 身份驗證
2. 指定任務組別名稱
3. 取得指定任務及長度的資料

---

### (0) 初始化 KGBuilder
載入初始 config 參數、octoparse 串接模組


In [7]:
from glob import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import pandas as pd
from tqdm import tqdm
from KGBuilder.config import *
from KGBuilder import octoparse
from KGBuilder.data_utils import colclean_time

初始參數如下：

其中，我們允許用戶在 `account_args` 修改 Octoparse 的登入帳密，當帳戶資訊改變時，無需改動模組依然可照常使用

In [9]:
args
gcs_args
data_args
account_args
model_args

account_args["octoparse_username"]
account_args["octoparse_password"]

{'root_dir': '/home/abao.yang@cathayholdings.com.tw/forDevTest/test_finKG',
 'article_dir': 'article_data',
 'model_dir': 'model_data',
 'is_china': 'False',
 'num_classes': 8,
 'max_len': 128,
 'start_date': '2022-05-09',
 'end_date': '2022-05-10',
 'timestamp': '20220613'}

{'service_account_key': '/home/abao.yang@cathayholdings.com.tw/forResearch/data/gcs/dst-dev2021-688422080376.json',
 'gcs_bucket_name': 'dst-financial-knowledge-graph',
 'gcs_dict_directory': 'dictionary',
 'gcs_model_directory': 'kgbuilder_models',
 'local_model_dir_baseBert': 'bert-base-chinese',
 'local_model_dir_FiNER': 'FiNER',
 'local_model_dir_FinRE': 'FinRE'}

{'article_filename': 'AllNews-20220613.csv',
 'cleansed_filename': 'cleansed-AllNews-20220613.csv',
 'module_EventTri_input_filename': 'event_triplets-input-20220613.json',
 'module_EventTri_output_filename': 'event_triplets-output-20220613.json',
 'module_FiNER_input_filename': 'event_triplets-output-20220613.json',
 'module_FiNER_output_filename': 'entity_extraction-output-20220613.json',
 'module_FinRE_input_filename': 'finre-input-20220613.json',
 'module_FinRE_output_filename': 'finre-output-20220613.json',
 'companylist_filename': 'semiconductor_companies_tpex.txt'}

{'china_octoparse_username': 'cathayholdingsct',
 'china_octoparse_password': 'Jj70827406',
 'octoparse_username': 'cathaydst',
 'octoparse_password': 'Cathaydst@2789',
 'neo4j_hostname': '10.0.10.6',
 'neo4j_username': 'neo4j',
 'neo4j_password': 'neo4jj',
 'neo4j_database': 'eventkg0610'}

{'classes_schema': 'all_8_schemas.json',
 'predicted_filename': 'FinKG-company_relations-20220613'}

'cathaydst'

'Cathaydst@2789'

#### (1) 身份驗證

模組會自動套用上一步 config 檔中的帳密資訊
- `account_args["octoparse_username"]`
- `account_args["octoparse_password"]`


其中，DST 有採購 國際版 & 大陸版 Octoparse，我們使用 `is_china` 參數指定要取得哪個版本的 API Token
- `False`: 國際版
- `True`: 大陸版

In [ ]:
# Octoparse API 身份驗證

base_url, username, password = octoparse.loginInfo(is_china=False)
print(base_url, username, password)
api_headers = octoparse.getAccessToken(base_url, username, password)
print(api_headers)


#### (2) 指定任務組別名稱

列出所有 group name、選擇要抓取的任務所在的群組

In [ ]:
# 取得 Octoparse tasks from "MyGroup"

group_dict = octoparse.getTaskGroup(base_url, api_headers)
group_dict = {g["taskGroupName"]: g["taskGroupId"] for g in group_dict}
tasks = octoparse.getTaskId(
    base_url, api_headers, taskGroupId=str(group_dict["MyGroup"])
)
print(tasks)


#### (3) 取得指定任務及長度的資料


1. 從列出的 task name 中選取要抓取的，指定到 `runFetchTask` 中的 `task` 參數，其中 `task` 是一個 `dict` ，包含 `taskName` 及 `taskId`
2. 在 `runFetchTask` 中可以指定以下參數：
   - `task`:  任務資訊 `{"taskName": "...", "taskId": "..."}`
   - `offset_history`: 從哪一筆開始抓取
   - `size`: 每次獲取幾筆(Octoparse 的上限是1000)
   - `fetch_len`:  抓取資料的筆數
     - all: 全部抓取
     - 其他數字: 只抓幾筆

In [ ]:
task = tasks[0]

export_data = octoparse.runFetchTask(
    base_url, api_headers, task, 
    offset_history=0, size=500, fetch_len="all"
)

export_data
